In [ ]:
import numpy as np

In [ ]:
with open("mnist_data/X.npy", "rb") as f:
    x_train = np.load(f)[:1000]

with open("mnist_data/y.npy", "rb") as f:
    y_train = np.load(f)[:1000]

In [ ]:
# Mean Squared Error
class MSELoss:
    def __call__(self, x, y):
        assert y.shape == x.shape, "mat1 and mat2 be must have same shape"
        return np.mean(np.square(x - y))

In [ ]:
# Binary CrossEntropy Loss
class BCELoss:
    def __call__(self, x, y):
        pass

In [ ]:
# ReLU activation
class ReLU:
    def __call__(self, x):
        return np.where(x > 0, x, 0)

    def __repr__(self):
        return f"ReLU()"

In [ ]:
# Sigmoid activation
class Sigmoid:
    def __call__(self, x):
        max_x = np.max(x)
        return np.exp(x - max_x) / (1 + np.exp(x - max_x))

    def __repr__(self):
        return f"Sigmoid()"

In [ ]:
# build a Neuron
# a neuron has weights and biases
# and does a linear transformation using matrix multiplication


class Neuron:
    def __init__(self, in_features):
        self._w = np.random.rand(in_features, 1)
        self._b = np.random.rand(1, 1)

    def __repr__(self):
        return f"Neuron(w={self._w.shape},b={self._b.shape})"

    def __call__(self, x):
        return x @ self._w + self._b

In [ ]:
# find a way to define a Layer which can hold any number of neurons


class Layer:
    def __init__(self, in_features, no_of_neurons):
        self._in = in_features
        self._neurons = tuple(Neuron(in_features) for _ in range(no_of_neurons))

    def __repr__(self):
        return f"Layer(in_features = {self._in}, out_features = {len(self._neurons)})"

    def __call__(self, x):
        assert x.ndim == 2, "Input must be a 2D matrix"
        return np.concatenate(tuple(neuron(x) for neuron in self._neurons), axis=1)

In [ ]:
# Base class
class Base:
    def __call__(self, x):
        return self.forward(x)

    def __repr__(self):
        return str(vars(self))

    def parameters(self):
        for layer in vars(self).values():
            if isinstance(layer, Layer):
                for neuron in layer._neurons:
                    yield neuron._w
                    yield neuron._b

In [ ]:
# neural network using Layer
class Net(Base):
    def __init__(self, in_features):
        self.l1 = Layer(in_features, 3)
        self.l2 = Layer(3, 1)
        self.relu = ReLU()
        self.sig = Sigmoid()

    def forward(self, x):
        x = self.relu(self.l1(x))
        x = self.sig(self.l2(x))
        return x

In [ ]:
# Instantiate the Net class
model = Net(x_train.shape[1])

In [ ]:
model(x_train).shape

In [ ]:
for param in model.parameters():
    print(param.shape)

In [ ]:
mse = MSELoss()
loss = mse(model(x_train), y_train)

In [ ]:
loss